In [1]:
import numpy as np 
import pandas as pd

from data import RandomData, AmazonBooks, ToyData, MovieLensData
from clustering import ClusteringModel

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Data stuff 

In [2]:
ds = AmazonBooks(min_user_ratings=10).get_dataset(verbose=True)
train = ds['train']
val = ds['val']

loading preprocessed dataset from disk


In [4]:
print(train.keys())
train['user_product_ratings'].head()

dict_keys(['user_product_ratings', 'product_descriptions', 'product_reviews'])


,user_id,product_id,rating
0,8996,35623,5.0
1,8996,35623,5.0
2,9341,53491,3.0
3,5598,79465,5.0
4,7217,63097,5.0


In [3]:
# len(train['user_product_ratings'].groupby('user_id')['rating'].count())

grouped_reviews = train['user_product_ratings'].groupby('user_id')['user_id', 'product_id', 'rating']
#     .aggregate(\
#         {'user_id':['mean'], 'review':lambda x: list(x)})
# grouped_reviews .columns = ['product_id', 'review']

In [91]:
# grouped_reviews.groups.keys()

In [202]:
# c_model = ClusteringModel()
# c_model._initialize_users(train)
# 


In [98]:
print(u.ratings)
g

[(2910, 4.0), (13886, 5.0), (8848, 5.0)]


,user_id,product_id,rating
943,1,2910,4.0
10947,1,13886,5.0
10951,1,8848,5.0


In [203]:
c.add_user(u)
c.new_center

{2910: (4.0, 1), 8848: (5.0, 1), 13886: (5.0, 1)}

In [204]:
c.recenter()
c.center

{2910: 4.0, 8848: 5.0, 13886: 5.0}

## centroid small tests  

In [6]:
from clustering import *

g = grouped_reviews.get_group(1)

c_model = ClusteringModel()
u = c_model.user(user_id= 0, grouped_data=g)
c = c_model.centroid(centroid_id = 0, initial_center={})

c.add_user(u)
c.recenter()

u2 =  c_model.user(user_id= 1, grouped_data=grouped_reviews.get_group(2))

print(c.center)
print(u.ratings)

c_model._distance(u, c)

{19912: 5.0, 127469: 5.0, 133186: 5.0, 37665: 1.0, 94620: 1.0, 39007: 2.0, 38240: 5.0}
{19912: 5.0, 127469: 5.0, 133186: 5.0, 37665: 1.0, 94620: 1.0, 39007: 2.0, 38240: 5.0}


0.0

In [39]:
c_model._distance(u2, c)

just_centroid {8848, 2910, 13886}
just_user set()
0.0
0.0
8.12403840463596


0.08124038404635961

In [37]:
# will clear and add users u2, u
c.add_user(u2)
c.add_user(u)
c.recenter()
print(c.center)

{9511: 4.0, 9704: 3.0, 14354: 4.0, 9999: 4.0, 13520: 4.0, 3718: 5.0, 7601: 4.0, 4321: 5.0, 11129: 5.0, 2910: 4.0, 13886: 5.0, 8848: 5.0}


In [16]:
for k in u.ratings:
    print(k)

2910
13886
8848


In [175]:
c_model.clusters[1].center

{96: 4.0,
 7116: 5.0,
 7361: 5.0,
 8285: 4.0,
 9516: 4.0,
 10141: 5.0,
 11536: 5.0,
 14914: 5.0}

## k_means perfing

In [70]:
from clustering import *

c_model = ClusteringModel(num_iterations=2)

In [63]:
def time_target():
    c_model = ClusteringModel(num_iterations=2)
    c_model.fit(train)

import timeit
timeit.Timer(stmt=time_target).timeit(number=30)


8.69278100000065

In [67]:
# for 10 
timeit.Timer(time_target, 'gc.enable()').timeit(number=30)


9.443330999999489

In [69]:
# for 50 
timeit.Timer(time_target, 'gc.enable()').timeit(number=30)


17.352155999999923

In [71]:
# for 70
timeit.Timer(time_target, 'gc.enable()').timeit(number=30)


20.161965999999666

In [72]:
# for 973
timeit.Timer(time_target, 'gc.enable()').timeit(number=30)


649.8086430000003

## converting results from clustering to be part of the database object
Goal take in map {user_id: cluster} and update the way we store data

in 

In [5]:
from clustering import *

c_model = ClusteringModel(num_iterations=70)
c_model.fit(train)


## clustering movie lens

In [4]:
import numpy as np 
import pandas as pd

from data import RandomData, AmazonBooks, ToyData, MovieLensData
from clustering import ClusteringModel

%matplotlib inline
%load_ext autoreload
%autoreload 2

ds = AmazonBooks(min_user_ratings=10).get_dataset(verbose=True)
# c_model = ClusteringModel(num_iterations=70)
# c_model.fit(train)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
loading preprocessed dataset from disk


In [5]:
grouped_reviews = ds['train']['user_product_ratings'].groupby('user_id')['user_id', 'product_id', 'rating']


In [6]:
len(grouped_reviews.groups.keys())

9643

In [ ]:
c_model = ClusteringModel(num_clusters=100,num_iterations=200)
c_model.fit(ds['train'])

interation: 0
interation: 1
interation: 2
interation: 3
interation: 4


In [18]:
assignments_train = {u.user_id:u.assigned_cluster for u in c_model.users} 
centers = [c.center for c in c_model.clusters]

In [20]:
import pickle
pickle.dump(assignments_train, open( "Amazon10_train_user_assignments.p", "wb" ) )
pickle.dump(centers, open( "Amazon10_train_centers.p", "wb" ) )

 